In [2]:
import os
import pandas as pd
from pathlib import Path

In [3]:
colnames = [
    'State_Name',
    'Constituency_No',
    'Year',
    'Position',
    'Candidate',
    'Party',
    'Age',
    'Constituency_Name',
    'Vote_Share_Percentage',
    'Turncoat',
    'MyNeta_education',
    'TCPD_Prof_Main',
]

dirname = '/home/amey/mihir/from_source'
filelist = [
    'TCPD_AE_Andhra_Pradesh_2024-12-23.csv',
    'TCPD_AE_Bihar_2024-12-23.csv',
    'TCPD_AE_Chhattisgarh_2024-12-23.csv',
    'TCPD_AE_Gujarat_2024-12-23.csv',
    'TCPD_AE_Haryana_2024-12-23.csv',
    'TCPD_AE_Karnataka_2024-12-23.csv',
    'TCPD_AE_Kerala_2024-12-23.csv',
    'TCPD_AE_Madhya_Pradesh_2024-12-23.csv',
    'TCPD_AE_Maharashtra_2024-12-23.csv',
    'TCPD_AE_Punjab_2024-12-23.csv',
    'TCPD_AE_Rajasthan_2024-12-23.csv',
    'TCPD_AE_Tamil_Nadu_2024-12-23.csv',
    'TCPD_AE_Telangana_2024-12-23.csv',
    'TCPD_AE_Uttar_Pradesh_2024-12-23.csv',
    'TCPD_AE_West_Bengal_2024-12-23.csv',
]
files_readable = [os.path.exists(os.path.join(dirname, f)) for f in filelist]
if all(files_readable):
    print('All files are available.')
else:
    print('Some files are not available.')

All files are available.


In [4]:
def process(pathname: str) -> None:
    raw_data = pd.read_csv(pathname, low_memory=False, encoding='iso-8859-1')
    data = raw_data[(raw_data['Year'] >= 2013) & (raw_data['Position'] == 1)][colnames]
    data['seq'] = data.groupby('Constituency_No')['Year'].rank('dense')
    winners = data[['Constituency_No', 'Year', 'Party', 'Position']][data['Position'] == 1].sort_values(by=['Constituency_No', 'Year'], ascending=False)
    winners['prev_winning_party'] = winners.groupby('Constituency_No')['Party'].shift(-1)
    data_1 = data.merge(winners[['Constituency_No', 'Year', 'prev_winning_party']], on=['Constituency_No', 'Year'], how='left')
    data_1['flag'] = data_1['Party'] == data_1['prev_winning_party']
    basename = Path(pathname).stem
    newname = basename.replace('TCPD_AE_', '').replace('_2024-12-23', '') + '_processed.csv'
    data_1.unique().to_csv(newname)

In [5]:
_ = [process(os.path.join(dirname, f)) for f in filelist]

AttributeError: 'DataFrame' object has no attribute 'unique'